# Boxed Types as Functions

When boxed types are used as functions instead of constructors, they do conversions.

In [81]:
(() => {
    console.log(Number('5')); // string to number
    console.log(typeof String(5));  // number to string
    console.log(Number('Infinity')); // special values
    console.log(Number('NaN'));
})();

5
string
Infinity
NaN


# Boxed Types as Constructors

It's almost the same thing as using them as functions except it __breaks pooling__ (makes equalities not work because you get a new object reference).

In [8]:
(() => {
    console.log(Number('5') === Number('5'));
    console.log(new Number('5') === new Number('5'));
})();

true
false


# Primitive Types as Functions/Constructors

The lowercase names don't function as consructors or functions.  They are only for `typeof` and for TypeScript annotations.

In [9]:
(() => {
    console.log(number('5')); // ILLEGAL
})();

ReferenceError: number is not defined

# Unary Plus and Minus

These operators convert their operand to `number`. This is equivalent to doing `Number()` or `-Number()`.

In [68]:
(() => {
    console.log(typeof +'5');
    console.log(typeof -'5');
    console.log(-'5');
    console.log();
})();

number
number
-5



# Built-In Functions

JS has built-in top-level functions like `parseInt`, `toString`, etc.

In [175]:
(() => {
    console.log(typeof parseInt('5'));
    console.log(typeof parseFloat('5'));
    console.log(typeof toString(5));
    
    console.log(parseInt('A', 16)); // hex for 10 in decimal
})();

number
number
string
10


# Invalid Number Conversions

Note that these __do not throw__.

Note that `parseInt` and `Number` differ in how they handle invalid chars in a number string.

In [212]:
(() => {
    console.log(parseInt('hi'));
    console.log(+'hi');
    console.log(Number('hi'));
    console.log(new Number('hi'));
    console.log(parseInt('5bla')); // stops after the 5
    console.log(Number('5bla'));   // NaN!
})();

NaN
NaN
NaN
[Number: NaN]
5
NaN


# Weird Cases of Numeric Conversions

Keep in mind this will also apply to __unary__ and __arithmetic__ operators that convert to numbers.

In [198]:
(() => {
    // arrays
    console.log(Number([])); // no elements -> 0
    console.log(Number([100])); // 1 element -> the element
    console.log(Number([100, 200])); // 2+ elements -> NaN
    
    // objects without conversions defined
    // (all NaN)
    console.log(Number({}));
    console.log(Number({a: 100}));
    console.log(Number(Number));
    console.log(Number(() => {}));
    
    // null and undefined
    console.log(Number(undefined)); // NaN
    console.log(Number(null)); // 0
    
    // booleans (not so weird)
    console.log(Number(true)); // 1
    console.log(Number(false)); // 0
    
    // empty
    console.log(Number("")); // 0
    console.log(Number()); // 0
    
    // other ways
    console.log(+[]); // 0
    console.log(3 + []); // 3
})();

0
100
NaN
NaN
NaN
NaN
NaN
NaN
0
1
0
0
0
0
3


# toString() on Built-In Types

The built-in types have string conversions via overriding `Object.toString()`.

Note that `console.log()` __does not use__ this because it lets you actually inspect the object in devtools.

Also note that __custom objects and classes__ do not have a useful implementation of this method (just get `[object Object]` instead).

In [178]:
(() => {
    console.log((5).toString());
    console.log([1, 2, 3].toString());
    console.log([1, undefined, 2].toString()); // undefined becomes "" instead of "undefined" in arrays
    console.log({x: 100}.toString());
    console.log({x: 100});
    console.log((true).toString());
    // console.log((null).toString()); // not allowed
    console.log([].toString()); // empty string (not [] as might expect)
})();

5
1,2,3
1,,2
[object Object]
{ x: 100 }
true



# Overriding toString()

In [26]:
(() => {
    const m = {x: 100, toString: function() { return `Here is x: ${this.x }`}};
    console.log(m);
    console.log(m.toString());
})();

{ x: 100, toString: [Function: toString] }
Here is x: 100


# valueOf()

The `valueOf()` method is meant to represent __conversion to a primitive__.

However, the base `Object.valueOf()` method returns `this` instead of a primitive, which defeats a lot of comparisons for objects that don't have this overriden.  __Built-in__ primitive types __override__ it (but things like arrays do not), which you can also do in your __custom objects__.

This is used more for __implicit coercions__ than ever actually called explicitly.

In [40]:
(() => {
    // base implementation
    const m = {};
    console.log(m.valueOf());
    console.log(m.valueOf() === m); // the object itself
    console.log();
    
    // built-in types
    console.log(typeof (5).valueOf());
    console.log(typeof "hi".valueOf());
    console.log(typeof [1, 2, 3].valueOf());
    const a = [1, 2, 3];
    console.log(a === a.valueOf()); // the object itself
    console.log();
    
    // custom types
    const n = {a: 100, b: 200, valueOf: () => 7};
    console.log(n);
    console.log(n.valueOf()); // number
    const o = {a: 100, b: 200, valueOf: () => 'hi'};
    console.log(o.valueOf()); // string
    console.log();
    
    // making your own boxed type (from MDN example)
    class Box {
        #value;
        
        constructor(value) {
            this.#value = value;
        }
        
        valueOf() {
            return this.#value;
        }
    }
    const b = new Box(15);
    const c = new Box('hi');
    console.log(b.valueOf());
    console.log(c.valueOf());
    
    // use in implicit conversions
    console.log(+b);
})();

{}
true

number
string
object
true

{ a: 100, b: 200, valueOf: [Function: valueOf] }
7
hi

15
hi
15


# Symbol.toPrimitive

You can provide `[Symbol.toPrimitive]` on a custom class to centralize the implicit conversion behavior.

It has __higher priority__ than `valueOf()` and `toString()`.  It is __not used by those__ functions if they are called directly.

Although it takes a __type hint__, you can __ignore it__ if you want, or respect it.

__The possible values of the hint string are as follows:__ (these will be discussed below)

1. `"number"` - for numeric coercion
1. `"string"` - for string coercion
1. `"default"` - for primitive coercion

In [46]:
(() => {
    class MyClass {
        [Symbol.toPrimitive](hint) {
            if (hint === 'number') {
                return 100;
            }
            else if (hint === 'string') {
                return 'hi';
            }
            else {
                return 404;
            }
        }
    }
    const m = new MyClass();
    
    console.log(m.toString()); // toString() doesn't use Symbol.toPrimitive implementation at all
    console.log(+m); // numeric coercion
    console.log(String(m)); // string coercion
    console.log(3 + m); // primitive coercion
})();

[object Object]
100
hi
407


# Primitive Coercion

This occurs when a primitive is needed, but when it __does not matter which type__.

__NOTE__: remember that __strings are primitives__.

__Algorithm__:

1. If __already a primitive__, use itself.
1. If `[Symbol.toPrimitive]` is defined, call it with a hint of `default` and use what is returned.
   - if it's not a primitive, throw `TypeError`
   - you can think of __arrays__ as doing this step for simplicity (`Number()` or `String()` depending on hint)(default to `String()`)
1. If `valueOf()` returns a primitive, use that.
1. If `toString()` returns a primitive (which it will by default), use that.
1. Otherwise, throw `TypeError`.

__Used By__:

1. `+` operator (when neither operand is a string)
1. `==` when the other operand is a __primitive__
1. API constructors (eg. `Date`) that can take multiple types of params

In [79]:
(() => {
    const m = {valueOf() {return 100}};
    const n = {valueOf() {return "hi"}};
    const o = {toString() { return "yo"}};
    
    console.log(3 + m); // primitive coercion -> step 3 (with a number)
    console.log(3 + n); // primitive coercion -> step 3 (with a string)
    console.log(3 + o); // primitive coercion -> step 4 (string)
    console.log([] + m); // primitive coercion -> array and step 3
    console.log([] + n); // primitive coercion -> array and step 3
})();

103
3hi
3yo
100
hi


# Numeric Coercion

This occurs when any operand of an __arithmetic operator__ is not numeric, unless it's going to become a string instead (will discuss individual operators later).  It's also used if you directly convert like with `Number()`.

The __algorithm__ is as follows:

1. If already a `number` or `bigint`, use itself.
1. Directly convert other primitives as appropriate (`boolean`, `undefined`, `null`, `bigint`, and `string`).
1. The rest is like steps 2 and above for __primitive coercion__ but with a type hint of `number`.
   - Instead of directly returning the first primitive we get, we __convert it to number__ (so could be 2nd level)

Certain places might be even more specific to only allow `number` or `bigint` in particular, such as array slice indices.  These will throw `TypeError` if so.  `bigint` coercion specifically has different rules (see MDN if needed).

In [55]:
(() => {
    const m = {valueOf() {return 100}};
    const n = {valueOf() {return "hi"}};
    const o = {toString() { return "yo"}};
    
    console.log(m - 99); // 1
    console.log(m / 2);  // 50
    console.log(m + 3); // 103
    
    console.log(n - 99);
})();

1
50
103
NaN


# String Coercion

The __algorithm__ is as follows:

1. If already a `string`, return it.
1. Directly convert other primitives as appropriate (`boolean`, `undefined`, `null`, `number`, `bigint`).
1. Try `[Symbol.toPrimitive]` with a type hint of `string` to get a primitive.  If not a string, convert it to string.
1. Try `toString()` next.
1. Finally try `valueOf()`.
   - note that the last 2 steps are __switched__ from how primitive and numeric conversions work
   
__Use Cases__:

1. Templates (backtick quote interpolation)
1. `String()` conversion
1. `+` operator when strings are present (see more detail in discussion of that operator below)

In [89]:
(() => {
    class MyClass {
        toString() {
            return 'yo';
        }
    }
    const m = new MyClass();
    
    console.log(`${m}`);
    console.log(String(m));
    console.log('hi' + m);
})();

yo
yo
hiyo


# Boolean Coercion

This is another way of saying __truthy__ vs. __falsey__ and can be checked with `!!` or `Boolean()`.

# Object Coercion (aka Boxing)

__Algorithm__:

- Objects returned as-is
- Primitives get boxed to their boxed types.
- `undefined` and `null` throw.

__Doing it Explicitly__:

- `Object.valueOf()` (the default implementation if didn't override)
- `Object()` (but doesn't throw for null and undefined)

__Implicit Use__: any place that passes an object as a param or uses it in a construct

   - eg. passing an integer to `Object.keys()` or using a string as the target of a `for` loop

In [91]:
(() => {
    console.log(Object.keys(5));
    for (const element in "hi") {
        console.log(element);
    }
})();

[]
0
1


# Binary + Operator

A lot of the complications with `+` come down to the fact that it is trying to be __two things__:

1. Arithmetic plus
1. String concatenation

Here's the __algorithm__ it uses:

1. Do __primitive coercion__ on both operands
1. If one side is `NaN` and the other is not a `number`, do __string coercion__ on both original operands instead
   - The docs don't mention this, but it's definitely happening if you test it
1. If only one side is a __string__ directly convert the other to a string and do a __concat__
1. Otherwise, if both sides are `bigint`, do __addition__
1. Directly convert both sides to `number` and do __addition__

In [114]:
(() => {
    console.log([] + 3); // 0 + 3 = 3 (addition)
    console.log({valueOf: () => NaN, toString: () => 'low'} + 3); // NaN + 3 = NaN (addition)
    console.log(undefined + 3); // undefined + 3 = NaN + 3 = NaN (addition)
    console.log(null + 3); // null + 3 = 0 + 3 = 3 (addition)
    console.log();
    
    console.log({valueOf: () => 'hi'} + 3); // 'hi' + 3 = 'hi' + '3' = 'hi3' (concat)
    console.log([1, 2, 3] + 4); // "1,2,3" + 4 = "1,2,3" + "4" = "1,2,34" (concat)
    console.log(NaN + "hi"); // NaN + "hi" =(re-convert) "NaN" + "hi" = NaNhi (concat)
    console.log({valueOf: () => NaN} + {toString: () => "ananana"}); // NaNananana
    console.log({} + []); // ""[object Object]" + "" = "[object Object]" (concat)
})();

3
NaN
NaN
3

hi3
1,2,34
NaNhi
NaNananana
[object Object]


# Other Arithmetic Operators

These try to __convert both operands__ to numbers and then give the result.  If you try to mix types, you will end up with `NaN`.

In [121]:
(() => {
    console.log('hi' - 3);
    console.log('hi' - 'low');
    
    console.log({} / 3);
    
    console.log([] / 3); // the only one here that is not NaN!
})();

NaN
NaN
NaN
0


## Loose Equality (== and !=)

The difference from the (usually better) __strict equality (=== and !==)__ is that it tries to __convert__ both sides to be directly comparable instead of requiring that they be the same type.

The __algorithm__ is as follows:

1. If they are the __same type__, directly compare their values just like `===` (references for objects)
   - Note that same type means __objects are all of the same type__ (two classes will compare with each other as references)
1. If one side is __nullish__, the other side must also be __nullish__
1. If one side is an object and the other is a primitive, do __primitive coercion__ on the object
1. At this point, __both are primitives__ but may be of different types
1. If they are the __same type__, do a direct comparison like step 1
1. If only one side if `boolean`, convert it to a number and start over
1. If a `string` and `number`, convert the string to a number
1. If both numeric, do a numeric comparison
1. If a `string` and `bigint`, convert the string to a bigint

__NOTE__: It is not true that `==` always does string comparison like some sources say, but it does feel it has a stronger preference for strings due to the `string` and `number` step in the algorithm.

In [195]:
(() => {
    console.log(5 == 5); // true
    console.log(5 == 'hi'); // false
    console.log(5 === 'hi'); // false
    console.log();
    
    console.log(null === undefined); // false
    console.log(null == undefined); // true (both nullish)
    console.log(undefined == 5); // false
    console.log();
    
    console.log({a: 100} == {a: 100}); // false (reference compare)
    console.log({a: 100} === {a: 100}); // false (reference compare)
    console.log();
    
    console.log({} == "[object Object]"); // true
    console.log({} === "[object Object]") // false (strict)
    console.log();
    
    console.log(1 === true); // false
    console.log(1 == true); // true
    console.log(true == "true"); // false (ouch!)
    console.log(true == 1); // true
    console.log();
    
    console.log(1 == "1"); // true
    console.log(1 === "1"); // false
    console.log();
    
    console.log([] == ""); // true
    console.log([] == {}); // false
    console.log([1] == 0); // false (because [1] defaults to "1" instead of 0)
    console.log()
    
    console.log({valueOf: () => 100} == 100); // true
    console.log({valueOf: () => "hi"} == "hi"); // true
    console.log({valueOf: () => "hi"} === "hi"); // false (strict)
    console.log({valueOf: () => "100"} == 100); // true
    console.log({valueOf: () => "100"} == "100"); // true
    console.log();
    
    console.log(0 == ""); // true (string becomes number)
})();

true
false
false

false
true
false

false
false

true
false

false
true
false
true

true
false

true
false
false

true
true
false
true
true

true


# Comparison Operators

Comparison operators (`<` and `>`) do the following:

1. Do __primitive coercion__ on both sides
1. If both sides are __strings__, do __lexographic comparison__
1. Otherwise, convert both to __number__ and do __numerical comparison__

__NOTE__: unlike `+` and `==` which seem to prefer strings, this seems to prefer numbers (due to the above).

Operators `<=` and `>=` instead do the following (with some edge case exceptions):

1. Swap the operands
1. Remove the `=` part from the operator
1. Negate the result

For instance, `x <= y` becomes `!(y < x)`.

It's also worth noting that coercion happens in __operand order__ (in case of side effects).

for instance, `x <= y` will coerce __x before y__ even though it becomes `!(y < x)` internally.

In [154]:
(() => {
    console.log("5" < 3); // 5 < 3 (false)
    console.log("3" < 5); // 3 < 5 (true)
    console.log("11" < 2); // 11 < 2 (false)
    console.log("11" < "2"); // "11" < "2" (true)
})();

false
true
false
true


# Operator Overloading

JavaScript __doesn't have__ true operator overloading like Python and C++ do, but by using `valueOf`, `toString`, and `Symbol.toPrimitive` you can exert some control over conversion, arithmetic, comparison, and boolean operations in a __similar way__.

In [158]:
(() => {
    class MyClass {
        x;
        
        constructor(val) {
            this.x = val;
        }
        
        valueOf() {
            return this.x;
        }
        
        toString() {
            return "x: " + this.x;
        }
    }
    const m = new MyClass(100);
    
    console.log(m == 100); // like we overloaded ==
    console.log(m === 100); // but only if we don't use strict
    console.log(m + 105); // like we overloaded +
    console.log(m + "[end]"); // like we overloaded String()
})();

true
false
205
100[end]


# Null-Prototype Objects

It is possible to have an object that inherits from `null` instead of from `Object.prototype` by manually hacking it.  You will get weird behavior, but it's worth noting in case you see it in the wild. Some API methods may even do it, but the public API is what matters.

# Popular Quirky Cases

In [215]:
(() => {
    // Math.min() -> Infinity
    // Math.max() -> -Infinity
    console.log(Math.min() < Math.max());
    console.log(Math.max() < Math.min());
    console.log();
    
    // boolean to number coercion
    // this will be the case in a lot of languages though
    // python is an exception because of chained comparisons
    console.log(3 > 2 > 1); // false
    console.log();
    
    // well-known bug retained for compatibility
    console.log(typeof null);
    console.log(typeof undefined);
    console.log();
    
    // "null" in base-24 -> all the letters map to digits
    console.log(parseInt(null, 24));
    console.log();
    
    // specified by IEEE actually
    console.log(NaN === NaN); // false
    
    // not that unexpected actually
    console.log([, , ,].toString());
    console.log();
    
    // beware of extra params you usually ignore!
    // map() callback actually takes 3
    console.log(['1', '7', '11'].map(parseInt)); // [1, NaN, 3]
    console.log(['1', '7', '11'].map(Number)); // this one works!
    console.log();
    
    // HTML-style comments in JS code
    <!-- This is a comment -->
    
    // boolean to number conversion
    console.log(true + true + true); // 3
    console.log();
    
    // comparison weirdness
    console.log(null > 0); // false
    console.log(null == 0); // false
    console.log(null >= 0); // true (!(0 > null) = !(false) = true
    console.log();
    
    // array weirdness
    console.log([] == ![]); // [] == false -> "" == 0 -> 0 == 0 -> true
    console.log();
    console.log();
    
    // number conversion weirdness
    console.log(parseInt(0.00000005)); // parseInt('5e-8') = 5
})();

false
true

false

object
undefined

23

false
,,

[ 1, NaN, 3 ]
[ 1, 7, 11 ]

3

false
false
true

true


5


# Weird JavaScript Generator

https://bluewings.github.io/en/writing-a-sentence-without-using-the-alphabet/#weird-javascript-generator

https://jsfuck.com/

Basic Ideas Behind This:

1. Using only `[]`, `+`, `!`, and `()` as primitive symbols
1. You can get `true` and `false` from `!![]` and `![]` due to `[]` being truthy
1. You can then get 0 and 1 with `+![]` and `+!![]` due to conversion of booleans to numbers with __unary plus__
1. You can get `undefined` from `[][[]]` from dereferencing an empty array (reification) using 0 index (array to number)
1. You can get `NaN` from `+{}` by direct conversion
1. You can get `Infinity` by dividing 1 by 0 (per the above definitions)
1. You can get `.` operator behavior using `[]` with a string in between
1. You can get `[object Object]` using `[] + {}`
1. You can get chars in that string adding 1s together and then put it in `[]` after `[] + {}`
   - You can often eliminate extra chars by making use of binary `+` behavior
1. You can get chars not in that string by getting other strings from the operator coercion behavior and indexing those
   - eg. use `NaN` to get an `a`
1. You can make whole strings by concatenating with `+`
1. You can wrap a string in `[]["filter"]["constructor"](YOURSTRING)()` to execute code
   - `[]["filter"]["constructor"]` means `[].filter.constructor` which is just a way to get the global `Function` constructor
   - the global `Function` constructor is usable as both a constructor and just a function
   - you can call that function with code as a string to get a new function and then call with args

In [204]:
(() => {
    console.log((![]+[])[+[]]+([][[]]+[])[+[]]+([][[]]+[])[+!![]]); // fun
})();

fun


In [205]:
[][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]][([][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]]+[])[!+[]+!+[]+!+[]]+(!![]+[][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]])[+!+[]+[+[]]]+([][[]]+[])[+!+[]]+(![]+[])[!+[]+!+[]+!+[]]+(!![]+[])[+[]]+(!![]+[])[+!+[]]+([][[]]+[])[+[]]+([][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]]+[])[!+[]+!+[]+!+[]]+(!![]+[])[+[]]+(!![]+[][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]])[+!+[]+[+[]]]+(!![]+[])[+!+[]]]((!![]+[])[+!+[]]+(!![]+[])[!+[]+!+[]+!+[]]+(!![]+[])[+[]]+([][[]]+[])[+[]]+(!![]+[])[+!+[]]+([][[]]+[])[+!+[]]+(+[![]]+[][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]])[+!+[]+[+!+[]]]+(!![]+[])[!+[]+!+[]+!+[]]+(+(!+[]+!+[]+!+[]+[+!+[]]))[(!![]+[])[+[]]+(!![]+[][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]])[+!+[]+[+[]]]+([]+[])[([][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]]+[])[!+[]+!+[]+!+[]]+(!![]+[][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]])[+!+[]+[+[]]]+([][[]]+[])[+!+[]]+(![]+[])[!+[]+!+[]+!+[]]+(!![]+[])[+[]]+(!![]+[])[+!+[]]+([][[]]+[])[+[]]+([][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]]+[])[!+[]+!+[]+!+[]]+(!![]+[])[+[]]+(!![]+[][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]])[+!+[]+[+[]]]+(!![]+[])[+!+[]]][([][[]]+[])[+!+[]]+(![]+[])[+!+[]]+((+[])[([][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]]+[])[!+[]+!+[]+!+[]]+(!![]+[][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]])[+!+[]+[+[]]]+([][[]]+[])[+!+[]]+(![]+[])[!+[]+!+[]+!+[]]+(!![]+[])[+[]]+(!![]+[])[+!+[]]+([][[]]+[])[+[]]+([][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]]+[])[!+[]+!+[]+!+[]]+(!![]+[])[+[]]+(!![]+[][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]])[+!+[]+[+[]]]+(!![]+[])[+!+[]]]+[])[+!+[]+[+!+[]]]+(!![]+[])[!+[]+!+[]+!+[]]]](!+[]+!+[]+!+[]+[!+[]+!+[]])+(![]+[])[+!+[]]+(![]+[])[!+[]+!+[]])()([][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]][([][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]]+[])[!+[]+!+[]+!+[]]+(!![]+[][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]])[+!+[]+[+[]]]+([][[]]+[])[+!+[]]+(![]+[])[!+[]+!+[]+!+[]]+(!![]+[])[+[]]+(!![]+[])[+!+[]]+([][[]]+[])[+[]]+([][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]]+[])[!+[]+!+[]+!+[]]+(!![]+[])[+[]]+(!![]+[][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]])[+!+[]+[+[]]]+(!![]+[])[+!+[]]]((!![]+[])[+!+[]]+(!![]+[])[!+[]+!+[]+!+[]]+(!![]+[])[+[]]+([][[]]+[])[+[]]+(!![]+[])[+!+[]]+([][[]]+[])[+!+[]]+([]+[])[(![]+[])[+[]]+(!![]+[][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]])[+!+[]+[+[]]]+([][[]]+[])[+!+[]]+(!![]+[])[+[]]+([][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]]+[])[!+[]+!+[]+!+[]]+(!![]+[][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]])[+!+[]+[+[]]]+(![]+[])[!+[]+!+[]]+(!![]+[][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]])[+!+[]+[+[]]]+(!![]+[])[+!+[]]]()[+!+[]+[!+[]+!+[]]]+((!![]+[])[+[]]+[+!+[]]+[!+[]+!+[]+!+[]+!+[]]+[!+[]+!+[]+!+[]]+(!![]+[])[+[]]+[+!+[]]+[!+[]+!+[]+!+[]+!+[]+!+[]]+[!+[]+!+[]+!+[]+!+[]+!+[]+!+[]+!+[]]+([][[]]+[])[+!+[]]+(![]+[])[!+[]+!+[]+!+[]]+(!![]+[])[+[]]+[+!+[]]+[!+[]+!+[]+!+[]+!+[]+!+[]]+[!+[]+!+[]+!+[]+!+[]+!+[]+!+[]+!+[]]+(![]+[])[!+[]+!+[]]+(!![]+[])[!+[]+!+[]+!+[]]+(+(+!+[]+[+!+[]]+(!![]+[])[!+[]+!+[]+!+[]]+[!+[]+!+[]]+[+[]])+[])[+!+[]]+(![]+[])[!+[]+!+[]]+(!![]+[])[+[]]+[+!+[]]+[!+[]+!+[]+!+[]+!+[]+!+[]]+[!+[]+!+[]+!+[]+!+[]+!+[]+!+[]+!+[]]+(!![]+[])[+[]]+[+!+[]]+[!+[]+!+[]+!+[]+!+[]]+[!+[]+!+[]+!+[]+!+[]+!+[]+!+[]+!+[]]+(!![]+[])[+[]]+[!+[]+!+[]+!+[]+!+[]+!+[]]+[+[]]+(!![]+[])[+[]]+[!+[]+!+[]+!+[]+!+[]]+[!+[]+!+[]+!+[]+!+[]+!+[]+!+[]+!+[]]+(!![]+[])[+[]]+[+!+[]]+[!+[]+!+[]+!+[]+!+[]+!+[]]+[+[]]+(!![]+[])[!+[]+!+[]+!+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[!+[]+!+[]]+(!![]+[])[+[]]+[+!+[]]+[!+[]+!+[]+!+[]+!+[]+!+[]]+[!+[]+!+[]+!+[]+!+[]+!+[]+!+[]+!+[]]+(!![]+[])[+[]]+[!+[]+!+[]+!+[]+!+[]]+[!+[]+!+[]+!+[]+!+[]+!+[]+!+[]+!+[]]+(!![]+[])[+[]]+[!+[]+!+[]+!+[]+!+[]+!+[]]+[+!+[]]+(!![]+[])[+[]]+[!+[]+!+[]+!+[]+!+[]+!+[]+!+[]+!+[]]+[!+[]+!+[]+!+[]])[(![]+[])[!+[]+!+[]+!+[]]+(+(!+[]+!+[]+[+!+[]]+[+!+[]]))[(!![]+[])[+[]]+(!![]+[][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]])[+!+[]+[+[]]]+([]+[])[([][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]]+[])[!+[]+!+[]+!+[]]+(!![]+[][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]])[+!+[]+[+[]]]+([][[]]+[])[+!+[]]+(![]+[])[!+[]+!+[]+!+[]]+(!![]+[])[+[]]+(!![]+[])[+!+[]]+([][[]]+[])[+[]]+([][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]]+[])[!+[]+!+[]+!+[]]+(!![]+[])[+[]]+(!![]+[][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]])[+!+[]+[+[]]]+(!![]+[])[+!+[]]][([][[]]+[])[+!+[]]+(![]+[])[+!+[]]+((+[])[([][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]]+[])[!+[]+!+[]+!+[]]+(!![]+[][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]])[+!+[]+[+[]]]+([][[]]+[])[+!+[]]+(![]+[])[!+[]+!+[]+!+[]]+(!![]+[])[+[]]+(!![]+[])[+!+[]]+([][[]]+[])[+[]]+([][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]]+[])[!+[]+!+[]+!+[]]+(!![]+[])[+[]]+(!![]+[][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]])[+!+[]+[+[]]]+(!![]+[])[+!+[]]]+[])[+!+[]+[+!+[]]]+(!![]+[])[!+[]+!+[]+!+[]]]](!+[]+!+[]+!+[]+[+!+[]])[+!+[]]+(![]+[])[!+[]+!+[]]+([![]]+[][[]])[+!+[]+[+[]]]+(!![]+[])[+[]]]((!![]+[])[+[]])[([][(!![]+[])[!+[]+!+[]+!+[]]+([][[]]+[])[+!+[]]+(!![]+[])[+[]]+(!![]+[])[+!+[]]+([![]]+[][[]])[+!+[]+[+[]]]+(!![]+[])[!+[]+!+[]+!+[]]+(![]+[])[!+[]+!+[]+!+[]]]()+[])[!+[]+!+[]+!+[]]+(!![]+[][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]])[+!+[]+[+[]]]+([![]]+[][[]])[+!+[]+[+[]]]+([][[]]+[])[+!+[]]](([][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]][([][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]]+[])[!+[]+!+[]+!+[]]+(!![]+[][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]])[+!+[]+[+[]]]+([][[]]+[])[+!+[]]+(![]+[])[!+[]+!+[]+!+[]]+(!![]+[])[+[]]+(!![]+[])[+!+[]]+([][[]]+[])[+[]]+([][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]]+[])[!+[]+!+[]+!+[]]+(!![]+[])[+[]]+(!![]+[][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]])[+!+[]+[+[]]]+(!![]+[])[+!+[]]]((!![]+[])[+!+[]]+(!![]+[])[!+[]+!+[]+!+[]]+(!![]+[])[+[]]+([][[]]+[])[+[]]+(!![]+[])[+!+[]]+([][[]]+[])[+!+[]]+(![]+[+[]])[([![]]+[][[]])[+!+[]+[+[]]]+(!![]+[])[+[]]+(![]+[])[+!+[]]+(![]+[])[!+[]+!+[]]+([![]]+[][[]])[+!+[]+[+[]]]+([][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]]+[])[!+[]+!+[]+!+[]]+(![]+[])[!+[]+!+[]+!+[]]]()[+!+[]+[+[]]]+![]+(![]+[+[]])[([![]]+[][[]])[+!+[]+[+[]]]+(!![]+[])[+[]]+(![]+[])[+!+[]]+(![]+[])[!+[]+!+[]]+([![]]+[][[]])[+!+[]+[+[]]]+([][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]]+[])[!+[]+!+[]+!+[]]+(![]+[])[!+[]+!+[]+!+[]]]()[+!+[]+[+[]]])()[([][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]]+[])[!+[]+!+[]+!+[]]+(!![]+[][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]])[+!+[]+[+[]]]+([][[]]+[])[+!+[]]+(![]+[])[!+[]+!+[]+!+[]]+(!![]+[])[+[]]+(!![]+[])[+!+[]]+([][[]]+[])[+[]]+([][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]]+[])[!+[]+!+[]+!+[]]+(!![]+[])[+[]]+(!![]+[][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]])[+!+[]+[+[]]]+(!![]+[])[+!+[]]]((![]+[+[]])[([![]]+[][[]])[+!+[]+[+[]]]+(!![]+[])[+[]]+(![]+[])[+!+[]]+(![]+[])[!+[]+!+[]]+([![]]+[][[]])[+!+[]+[+[]]]+([][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]]+[])[!+[]+!+[]+!+[]]+(![]+[])[!+[]+!+[]+!+[]]]()[+!+[]+[+[]]])+[])[+!+[]])+([]+[])[(![]+[])[+[]]+(!![]+[][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]])[+!+[]+[+[]]]+([][[]]+[])[+!+[]]+(!![]+[])[+[]]+([][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]]+[])[!+[]+!+[]+!+[]]+(!![]+[][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]])[+!+[]+[+[]]]+(![]+[])[!+[]+!+[]]+(!![]+[][(![]+[])[+[]]+(![]+[])[!+[]+!+[]]+(![]+[])[+!+[]]+(!![]+[])[+[]]])[+!+[]+[+[]]]+(!![]+[])[+!+[]]]()[+!+[]+[!+[]+!+[]]])())

hello
